# IKATS Python API

This Notebook is a tutorial about how to use the **IKATS Python API**.

In [ ]:
# Import IKATS Api
from ikats import IkatsAPI
from ikats.exceptions import *
from ikats.extra.timeseries import *

# Create the API instance connected to localhost
if False:
    api = IkatsAPI(host="http://preprod.intra.ikats.org", port=80)
    api.session.tdm_url = "/dev-datamodel"
    api.session.tsdb_url = "/dev-opentsdb"
else:
    api = IkatsAPI(host="http://localhost", port=80)

## Timeseries

This part shows how to :
- create a new Timeseries
- add points to it (using a random points generator)
- save it
- delete it

In [ ]:
# Create a local timeseries
ts = api.ts.new()
print("New TS locally created:", ts)
print("No TSUID is defined:", ts.tsuid)
print("No FID is defined:", ts.fid)
print("No points:", ts.data)

In [ ]:
# Adding some random points
ts.data = gen_random_ts(sd=1000000000000, ed=1000000010000, nb_points=10)

# Set a FID
ts.fid = "MyTS"
print("A FID is added:", ts.fid)
print("but still no TSUID:",ts.tsuid)
print("and the points are still there:", len(ts)) # You could also use `len(ts.data)`

In [ ]:
# Now we save the TS and request to generate minimum metadata (ikats_start_date, ikats_end_date, qual_nb_points)
ts.save(generate_metadata=True)

# Check what's new
print("The TSUID is now set to %s (because of the save action)" % ts.tsuid)

In [ ]:
# We can still add points locally
ts.data += gen_random_ts(sd=1000000011000, ed=1000000020000, nb_points=10)
print("The length of the TS has increased : %s" % len(ts))

# But we need to save them.
ts.save()

In [ ]:
# Deletion of the Timeseries
if ts.delete(raise_exception=False):
    print("The Timeseries has been deleted")

try:
    ts.delete(raise_exception=True)
except:
    # not working : see bugs #2738
    print("The Timeseries has already been deleted (first way to check it)")
    
if ts.delete(raise_exception=False):
    print("The Timeseries has already been deleted (second way to check it)")

In [ ]:
# Timeseries still locally present. Save it again (for other cases to cover)
ts.save()

## Metadata

In [ ]:
from ikats.lib import MDType

# Add specific metadata locally (corresponding to "generate_metadata" flag of `ts.save()`)
ts.metadata.set(name="qual_nb_points", value=len(ts), dtype=MDType.NUMBER)
ts.metadata.set(name="ikats_start_date", value=ts.data[0][0], dtype=MDType.NUMBER)
ts.metadata.set(name="ikats_end_date", value=ts.data[-1][0], dtype=MDType.NUMBER)

# Read metadata from local cache
print ("qual_nb_points=%d"%ts.metadata.get("qual_nb_points"))

# Save metadata
ts.metadata.save()

# Delete metadata from local cache
ts.metadata.set(name="test_md", value="42", dtype=MDType.STRING)
ts.metadata.delete(name="test_md")
try:
    ts.metadata.get(name="test_md")
except IkatsNotFoundError:
    print("This metadata doesn't exist")

# Overwrite local cache with remote database
ts.metadata.fetch()
print("Number of metadata %s" % len(ts.metadata))

# The metadata doesn't exist in remote but there is no way to know it locally.
# This action marks the metadata as deleted.
# Next time the metadata are saved, the deletion will occur on remote side
ts.metadata.delete(name="test_md")

## Dataset

In [ ]:
# Get the list of all Datasets
api.ds.list()

In [ ]:
# Create a Dataset
ds = api.ds.new(name="MyDataset")

In [ ]:
# Read a Dataset
print("Getting Dataset '%s'" % ds.name)
print("The description of the Dataset is '%s'" % ds.desc)
print("The Dataset is composed of %s Timeseries:  " % len(ds)) # You could also use `len(ds.ts)`
for ts in ds.ts:
    print("  - %s" % ts)

In [ ]:
# Save as a new Dataset
ds.name = "New_Name"
ds.description = "My new description"
try:
    api.ds.save(ds)
except ValueError:
    print("You shall add TS to dataset locally or you get a ValueError")

In [ ]:
try:
    ts_list = [api.ts.new(fid="FID_TEST_%s" % x) for x in range(10)]
    ds.ts = ts_list
except IkatsConflictError:
    print("You can't create a new TS with a FID that already exist")

try:
    api.ds.save(ds)
    print("Dataset saved")
except IkatsConflictError:
    print("Dataset already exist")

In [ ]:
# Delete the dataset
ds.name = "New_Name"
try:
    api.ds.delete(ds.name, deep=True)
    print("Dataset deleted. Associated TS are also deleted")
except IkatsNotFoundError:
    print("Dataset Not Found")